<a href="https://colab.research.google.com/github/Psousace/Fundos_Imobiliarios/blob/main/Dividendos_Fiis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl

class fii:
    def __init__(self, ticker):
        self.ticker = ticker
    def pegar_dividendos_fii(self):
        agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
        url = f"https://www.fundamentus.com.br/fii_proventos.php?papel={self.ticker}&tipo=2"
        webpage = requests.get(url, headers=agent)
        soup = BeautifulSoup(webpage.content, "html.parser")
        resultados = soup.find(id = "resultado")
        tabela_body = resultados.find("tbody")
        linhas = tabela_body.find_all("tr")
        data = []
        for linha in linhas:
            colunas = linha.find_all("td")
            colunas = [ele.text.strip() for ele in colunas]
            data.append([ele for ele in colunas if ele])
        data_output = pd.DataFrame(data, columns=["Data_com", "rendimento", "data_rendimento", "valor"])
        data_output['papel'] = self.ticker
        # aqui pegaremos sempre o ultimo dividendo disponivel
        data_output = data_output.iloc[0]

        return data_output

def get_dividend(papel):
    return fii(papel).pegar_dividendos_fii()

# usando map para rodar a funcao para os tickers
resultados_map = pd.DataFrame(list(map(get_dividend, ['BLMG11','BRCO11', 'BTLG11','GALG11', 'HGLG11' ,'HLOG11', 'LGCP11', 'LVBI11', 'NEWL11', 'PATL11', 'RBRL11', 'SDIL11', 'XPLG11' ])))

# exportando FIIs
#resultados_map.to_excel("dividendos_fiis.xlsx")
print(resultados_map)

     Data_com  rendimento data_rendimento valor   papel
0  07/08/2023  Rendimento      14/08/2023  0,70  BLMG11
0  31/08/2023  Rendimento      15/09/2023  0,87  BRCO11
0  15/08/2023  Rendimento      25/08/2023  0,76  BTLG11
0  31/08/2023  Rendimento      08/09/2023  0,08  GALG11
0  31/08/2023  Rendimento      15/09/2023  1,10  HGLG11
0  31/08/2023  Rendimento      15/09/2023  0,65  HLOG11
0  31/08/2023  Rendimento      15/09/2023  0,64  LGCP11
0  31/08/2023  Rendimento      08/09/2023  0,79  LVBI11
0  31/08/2023  Rendimento      15/09/2023  0,95  NEWL11
0  31/08/2023  Rendimento      11/09/2023  0,60  PATL11
0  07/08/2023  Rendimento      14/08/2023  0,65  RBRL11
0  30/06/2023  Rendimento      14/07/2023  0,83  SDIL11
0  31/08/2023  Rendimento      15/09/2023  0,78  XPLG11
